In [14]:
from collections import namedtuple, deque
from heapq import heappush, heappop
import numpy as np

In [15]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

In [16]:
def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

### Define Heuristic and A* Algorithm

In [19]:
# Heuristic function: Manhattan distance
def manhattan_distance(state: np.ndarray) -> int:
    distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value == 0: 
                continue
            goal_x, goal_y = divmod(value - 1, PUZZLE_DIM)
            distance += abs(x - goal_x) + abs(y - goal_y)
    return distance

# A* Search Algorithm
def a_star_search(initial: np.ndarray, goal: np.ndarray):
    priority_queue = []
    heappush(priority_queue, (0, initial.tobytes(), [], 0)) 
    visited = set()
    node_evaluated = 0
    
    while priority_queue:
        _, current_ser, path, cost = heappop(priority_queue)
        current = np.frombuffer(current_ser, dtype=initial.dtype).reshape(initial.shape)
        node_evaluated += 1
        
        if np.array_equal(current, goal):
            print(current)
            return path, node_evaluated  # Return solution path
        
        visited.add(current_ser)
        
        for action in available_actions(current):
            next_state = do_action(current, action)
            next_ser = next_state.tobytes()
            if next_ser in visited:
                continue
            
            new_cost = cost + 1
            priority = new_cost + manhattan_distance(next_state)
            heappush(priority_queue, (priority, next_ser, path + [action], new_cost))
    
    return None, None 

### Execution and Results

In [18]:
# An example of a possible initial state with n=4
initial_state = np.array([
    [1, 2, 3, 12],
    [11, 10, 9, 8],
    [7, 6, 5, 4],
    [13, 14, 15, 0]
])

goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
solution_path, tot_cost = a_star_search(initial_state, goal_state)

if solution_path:
    print(f"Solution found in {len(solution_path)} moves.")
    print(f"Total cost: {tot_cost}")
    print("Solution steps:")
    for step in solution_path:
        print(step)
else:
    print("No solution found.")

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]]
Solution found in 40 moves.
Total cost: 625970
Efficiency: 0.00
Solution steps:
Action(pos1=(3, 3), pos2=(3, 2))
Action(pos1=(3, 2), pos2=(3, 1))
Action(pos1=(3, 1), pos2=(2, 1))
Action(pos1=(2, 1), pos2=(2, 2))
Action(pos1=(2, 2), pos2=(1, 2))
Action(pos1=(1, 2), pos2=(1, 1))
Action(pos1=(1, 1), pos2=(1, 0))
Action(pos1=(1, 0), pos2=(2, 0))
Action(pos1=(2, 0), pos2=(2, 1))
Action(pos1=(2, 1), pos2=(2, 2))
Action(pos1=(2, 2), pos2=(1, 2))
Action(pos1=(1, 2), pos2=(1, 1))
Action(pos1=(1, 1), pos2=(0, 1))
Action(pos1=(0, 1), pos2=(0, 2))
Action(pos1=(0, 2), pos2=(0, 3))
Action(pos1=(0, 3), pos2=(1, 3))
Action(pos1=(1, 3), pos2=(2, 3))
Action(pos1=(2, 3), pos2=(2, 2))
Action(pos1=(2, 2), pos2=(1, 2))
Action(pos1=(1, 2), pos2=(0, 2))
Action(pos1=(0, 2), pos2=(0, 3))
Action(pos1=(0, 3), pos2=(1, 3))
Action(pos1=(1, 3), pos2=(1, 2))
Action(pos1=(1, 2), pos2=(0, 2))
Action(pos1=(0, 2), pos2=(0, 1))
Action(pos1=(0, 1), pos2=(1, 1))
A